In [6]:
import numpy as np
import os
data_path = 'speech_wav_8000.npz'
speech_data = np.load(data_path)

print("✅")

✅


In [7]:
print("Wave data shape : ", speech_data["wav_vals"].shape)
print("Label data shape : ", speech_data["label_vals"].shape)
print("✅")

Wave data shape :  (50620, 8000)
Label data shape :  (50620, 1)
✅


In [8]:
import IPython.display as ipd
import random
# 데이터 선택 (랜덤하게 선택하고 있으니, 여러번 실행해 보세요)
rand = random.randint(0, len(speech_data["wav_vals"]))
print("rand num : ", rand)
sr = 8000 # 1초동안 재생되는 샘플의 갯수
data = speech_data["wav_vals"][rand]
print("Wave data shape : ", data.shape)
print("label : ", speech_data["label_vals"][rand])
ipd.Audio(data, rate=sr)

rand num :  45529
Wave data shape :  (8000,)
label :  ['unknown']


In [10]:
target_list = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']
label_value = target_list
label_value.append('unknown')
label_value.append('silence')

print('LABEL : ', label_value)

new_label_value = dict()
for i, l in enumerate(label_value):
    new_label_value[l] = i
label_value = new_label_value

print('Indexed LABEL : ', new_label_value)

LABEL :  ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go', 'unknown', 'silence']
Indexed LABEL :  {'yes': 0, 'no': 1, 'up': 2, 'down': 3, 'left': 4, 'right': 5, 'on': 6, 'off': 7, 'stop': 8, 'go': 9, 'unknown': 10, 'silence': 11}


In [12]:
temp = []
for v in speech_data["label_vals"]:
    temp.append(label_value[v[0]])
label_data = np.array(temp)
label_data

array([ 3,  3,  3, ..., 11, 11, 11])

In [13]:
from sklearn.model_selection import train_test_split

sr = 8000
train_wav, test_wav, train_label, test_label = train_test_split(speech_data["wav_vals"],label_data,test_size=0.1,shuffle=True)

print(train_wav)

train_wav = train_wav.reshape([-1, sr, 1]) # add channel for CNN
test_wav = test_wav.reshape([-1, sr, 1])
print("✅")


[[ 3.4168279e-03  5.2273110e-03  5.9024006e-04 ...  2.2987102e-03
   1.6717988e-03 -2.3386294e-03]
 [ 6.2764422e-03  1.4127310e-02  1.0973018e-02 ... -5.4128999e-03
  -2.1037154e-03 -4.4694701e-03]
 [-4.1886424e-06 -1.1775300e-04 -2.3891548e-04 ...  4.3292824e-04
   4.8708063e-04  4.8978371e-04]
 ...
 [-1.5026439e-04 -1.0790041e-04  1.7209681e-04 ... -2.1528699e-03
  -1.5382017e-03  4.7543677e-04]
 [-6.3443679e-04 -3.1228100e-03 -3.6444440e-03 ... -4.2386455e-03
  -5.7402323e-03 -4.2056488e-03]
 [ 1.5354856e-03  2.0451983e-03  2.5023203e-03 ...  2.5339162e-03
   3.6731926e-03  5.1221834e-03]]
✅


In [14]:
print("train data : ", train_wav.shape)
print("train labels : ", train_label.shape)
print("test data : ", test_wav.shape)
print("test labels : ", test_label.shape)
print("✅")

train data :  (45558, 8000, 1)
train labels :  (45558,)
test data :  (5062, 8000, 1)
test labels :  (5062,)
✅


In [15]:
batch_size = 32
max_epochs = 10
# the save point
checkpoint_dir = 'C:/Users/User'
checkpoint_dir

'C:/Users/User'

In [17]:
def one_hot_label(wav, label):
    label = tf.one_hot(label, depth=12)
    return wav, label
print("✅")

✅


In [18]:
import tensorflow as tf
# for train
train_dataset = tf.data.Dataset.from_tensor_slices((train_wav, train_label))
train_dataset = train_dataset.map(one_hot_label)
train_dataset = train_dataset.repeat().batch(batch_size=batch_size)
print(train_dataset)
# for test
test_dataset = tf.data.Dataset.from_tensor_slices((test_wav, test_label))
test_dataset = test_dataset.map(one_hot_label)
test_dataset = test_dataset.batch(batch_size=batch_size)
print(test_dataset)
print("✅")

<BatchDataset shapes: ((None, 8000, 1), (None, 12)), types: (tf.float32, tf.float32)>
<BatchDataset shapes: ((None, 8000, 1), (None, 12)), types: (tf.float32, tf.float32)>
✅


In [20]:
from tensorflow.keras import layers
input_tensor = layers.Input(shape=(sr, 1))
x = layers.Conv1D(32, 9, padding='same', activation='relu')(input_tensor)
x = layers.Conv1D(32, 9, padding='same', activation='relu')(x)
x = layers.MaxPool1D()(x)
x = layers.Conv1D(64, 9, padding='same', activation='relu')(x)
x = layers.Conv1D(64, 9, padding='same', activation='relu')(x)
x = layers.MaxPool1D()(x)
x = layers.Conv1D(128, 9, padding='same', activation='relu')(x)
x = layers.Conv1D(128, 9, padding='same', activation='relu')(x)
x = layers.Conv1D(128, 9, padding='same', activation='relu')(x)
x = layers.MaxPool1D()(x)
x = layers.Conv1D(256, 9, padding='same', activation='relu')(x)
x = layers.Conv1D(256, 9, padding='same', activation='relu')(x)
x = layers.Conv1D(256, 9, padding='same', activation='relu')(x)
x = layers.MaxPool1D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
output_tensor = layers.Dense(12)(x)
model_wav = tf.keras.Model(input_tensor, output_tensor)
model_wav.summary()


Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 8000, 1)]         0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 8000, 32)          320       
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 8000, 32)          9248      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 4000, 32)          0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 4000, 64)          18496     
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 4000, 64)          36928     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 2000, 64)         

In [22]:
optimizer=tf.keras.optimizers.Adam(1e-4)
model_wav.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),optimizer=optimizer,metrics=['accuracy'])
print("✅")

✅


In [23]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_dir,save_weights_only=True,monitor='val_loss',mode='auto',save_best_only=True,verbose=1)
print("✅")

✅


In [24]:
history_wav = model_wav.fit(train_dataset, epochs=max_epochs,steps_per_epoch=len(train_wav) // batch_size,validation_data=test_dataset,validation_steps=len(test_wav) // batch_size,callbacks=[cp_callback])
print("✅")

Epoch 1/10
 135/1423 [=>............................] - ETA: 1:37:15 - loss: 2.2824 - accuracy: 0.2053

KeyboardInterrupt: 